In [34]:
import pandas as pd
df = pd.read_csv('../data/2021玉山人工智慧公開挑戰賽冬季賽訓練資料集.gz.gz', compression='gzip')
df.columns = ['dt'] + df.columns[1:].tolist() # 更改欄位名稱
test_data = pd.read_csv('..\data\需預測的顧客名單及提交檔案範例.csv')
print(train_data.shape,test_data.shape)
display(train_data.head())
display(test_data.head())

(32475654, 53) (500000, 4)


,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
0,1.0,10321418.0,45,3.0,3891.965283,3.0,0.0,0.0,0.0,1.00,...,1.0,4.0,5.0,1.0,99.0,30.0,95982.822967,1.0,4.0,1.0
1,1.0,10414574.0,15,2.0,10616.561549,2.0,0.0,0.0,0.0,1.00,...,2.0,2.0,15.0,1.0,2.0,30.0,130702.351368,1.0,3.0,1.0
2,1.0,10134567.0,48,2.0,23527.655416,0.0,2.0,0.0,0.0,0.00,...,1.0,3.0,11.0,1.0,3.0,30.0,112010.611717,0.0,4.0,1.0
3,1.0,10001003.0,48,9.0,17751.558260,7.0,2.0,0.0,0.0,0.75,...,2.0,3.0,11.0,1.0,2.0,30.0,59701.507360,0.0,3.0,1.0
4,1.0,10267183.0,2,1.0,21701.307598,1.0,0.0,0.0,0.0,1.00,...,2.0,6.0,15.0,1.0,99.0,30.0,NaN,0.0,6.0,0.0


,chid,top1,top2,top3
0,10128239,18,10,6
1,10077943,48,22,6
2,10277876,22,10,39
3,10364842,37,19,13
4,10392717,2,36,26


In [39]:
feature_columns = ['masts','educd','trdtp','naty','poscd','cuorg']
sample_submit = test_data
train_df = train_data

In [40]:
from tqdm import tqdm_notebook as tqdm
for idx in tqdm(sample_submit.index[:5]): # demo用所以取前五筆 實際上應該取全部
    for col in feature_columns:
        # 取出這一條row的chid
        chid = sample_submit.loc[idx,'chid']
        # 從歷史訓練數據抓對應的chid 算一些統計資訊 當作特徵 併入sample_submit裡面
        sample_submit.loc[idx,f"mean_of_{col}"] = train_df.loc[train_df['chid']==chid,col].mean()
        sample_submit.loc[idx,f"nunique_of_{col}"] = train_df.loc[train_df['chid']==chid,col].nunique()
sample_submit.head()

C:\Users\ricky\AppData\Local\Temp/ipykernel_7064/238154688.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx in tqdm(sample_submit.index[:5]):


,chid,top1,top2,top3,mean_of_masts,nunique_of_masts,mean_of_educd,nunique_of_educd,mean_of_trdtp,nunique_of_trdtp,mean_of_naty,nunique_of_naty,mean_of_poscd,nunique_of_poscd,mean_of_cuorg,nunique_of_cuorg
0,10128239,18,10,6,1.0,1.0,4.0,1.0,8.0,1.0,1.0,1.0,1.0,1.0,30.0,1.0
1,10077943,48,22,6,1.0,1.0,5.0,1.0,12.0,1.0,1.0,1.0,2.0,1.0,30.0,1.0
2,10277876,22,10,39,1.0,1.0,5.0,1.0,6.0,1.0,1.0,1.0,1.0,1.0,30.0,1.0
3,10364842,37,19,13,2.0,1.0,5.0,1.0,7.0,1.0,1.0,1.0,99.0,1.0,30.0,1.0
4,10392717,2,36,26,2.0,1.0,3.0,1.0,5.0,1.0,1.0,1.0,2.0,1.0,30.0,1.0


# 保存

In [41]:
train_data.reset_index().to_feather('../data/train_data.feather')
test_data.reset_index().to_feather('../data/test_data.feather')